Packages loading

In [24]:
import numpy as np
import pandas as pd
from scipy.stats import dirichlet, norm
import random

# AR(1) with uniform prior

## Data loading

In [ ]:
# 69 pays, longueur 27 ans (entre 1998 et 2024), normalisé entre 0 et 1
GDP = pd.read_csv('../Data preprocessing/GDP_yearly_length_27_normalized')

In [67]:
GDP_global = pd.read_csv('../Data preprocessing/GDP_yearly_27_global_normalized')

In [8]:
GDP = GDP.drop(GDP.columns[0], axis=1)

In [9]:
GDP.head()

,Advanced Economies,Argentina,Australia,Austria,Belgium,Bulgaria,Bolivia,Brazil,Botswana,Canada,...,Slovakia,Slovenia,Sweden,Thailand,Turkey,"Taiwan, China",Uruguay,United States,World (WBG members),South Africa
0,0.000000,0.465062,0.000000,0.000000,0.000000,0.078101,0.000000,0.000000,0.000000,0.000000,...,0.004087,0.000000,0.000000,0.000000,0.015166,0.000000,0.193577,0.000000,0.000000,0.000000
1,0.058263,0.419842,0.043460,0.075248,0.065662,0.000000,0.002935,0.006361,0.058236,0.071799,...,0.000000,0.059256,0.056431,0.036479,0.000000,0.043589,0.169978,0.063511,0.035017,0.032060
2,0.131093,0.409848,0.075163,0.148157,0.136995,0.036144,0.021531,0.065312,0.089006,0.147254,...,0.006259,0.107926,0.128310,0.073154,0.031237,0.087227,0.146532,0.120167,0.082889,0.089510
3,0.159332,0.352416,0.102593,0.176508,0.158882,0.070884,0.034685,0.086347,0.108492,0.176202,...,0.031499,0.145398,0.152396,0.102717,0.003282,0.076927,0.100792,0.133988,0.103997,0.127994
4,0.187174,0.218249,0.147604,0.209524,0.193231,0.124303,0.054367,0.130058,0.168572,0.223374,...,0.070471,0.187079,0.188187,0.157387,0.032078,0.116652,0.012801,0.158818,0.129927,0.182160


## Premier exemple avec p(yi∣θk) gaussienne 
(yi gaussienne de moyenne θk)

In [15]:
# --- Étape 1: Initialisation ---
def initialize_model(y, K, e0=1.0):
    """
    Initialise les paramètres du modèle.
    y: Liste des séries temporelles [N x T]
    K: Nombre de groupes
    e0: Hyperparamètre du Dirichlet
    """
    N, T = y.shape
    eta = dirichlet.rvs([e0] * K, size=1).flatten()  # Dirichlet prior for eta
    S = np.random.choice(K, size=N)  # Random initial classification
    theta = [np.random.randn(T) for _ in range(K)]  # Paramètres pour chaque groupe
    return eta, S, theta

In [16]:
# --- Étape 2: Échantillonnage de S ---
def sample_S(y, theta, eta):
    """
    Échantillonne les indicateurs de groupe S.
    """
    N = len(y)
    K = len(theta)
    S = np.zeros(N, dtype=int)
    for i in range(N):
        probs = [
            norm.pdf(y[i], loc=theta[k]).prod() * eta[k]  # Probabilité conditionnelle p(yi∣θk) gaussienne
            for k in range(K)
        ]
        probs = np.array(probs)
        probs /= probs.sum()  # Normalisation
        S[i] = np.random.choice(K, p=probs)  # Échantillonnage
    return S

In [17]:
# --- Étape 3: Mise à jour des paramètres ---
def update_theta(y, S, K):
    """
    Met à jour les paramètres spécifiques aux groupes (theta).
    """
    T = y.shape[1]
    theta = []
    for k in range(K):
        group_data = y[S == k]
        if len(group_data) > 0:
            theta_k = group_data.mean(axis=0)  # Posterior mean (simple estimation)
        else:
            theta_k = np.random.randn(T)  # Cas sans données
        theta.append(theta_k)
    return theta

def update_eta(S, K, e0=1.0):
    """
    Met à jour les paramètres eta (structure d'ignorance).
    """
    counts = np.bincount(S, minlength=K) + e0  # Comptes augmentés par le prior
    return dirichlet.rvs(counts, size=1).flatten()

In [20]:
# --- Étape 4: MCMC ---
def mcmc(y, K, num_iter=1000, e0=1.0):
    """
    Effectue l'estimation via MCMC.
    """
    eta, S, theta = initialize_model(y, K, e0)
    for iteration in range(num_iter):
        # Échantillonnage de S
        S = sample_S(y, theta, eta)
        # Mise à jour de theta
        theta = update_theta(y, S, K)
        # Mise à jour de eta
        eta = update_eta(S, K, e0)
    return S, theta, eta

In [ ]:
# Exécution pour des données synthétiques

# Génération de données synthétiques
np.random.seed(42)
N, T, K = 100, 50, 3
true_theta = [np.random.randn(T) for _ in range(K)]
y = np.vstack([np.random.randn(1, T) + true_theta[k] for k in np.random.choice(K, N)])

# Estimation MCMC
S_est, theta_est, eta_est = mcmc(y, K, num_iter=500)

print("Classification estimée : ", S_est)
print("Paramètres des groupes : ", theta_est)
print("Paramètres eta : ", eta_est)

Classification estimée :  [1 2 0 2 1 1 0 1 2 0 2 2 1 1 1 1 1 0 2 2 2 2 1 2 0 0 2 2 0 0 1 0 1 1 1 1 1
 2 0 1 0 0 1 2 1 2 0 2 2 2 0 0 2 2 1 2 1 2 1 0 2 2 2 2 1 2 0 0 2 0 2 2 0 2
 0 1 2 1 0 1 1 0 1 1 0 1 0 1 0 0 1 1 1 2 2 2 0 1 0 2]
Paramètres des groupes :  [array([-1.43885876, -0.28508876,  0.07423943, -0.56848096, -0.06911439,
        0.84361014,  1.71529438,  0.00278895, -0.04675652, -0.21411054,
       -1.91502398, -0.17995909,  0.53696021,  2.1140656 ,  0.09221716,
        0.31718667,  0.1964794 , -1.17957364,  0.94882226,  0.83776001,
        0.85822083, -0.91832784,  1.35476824, -1.33344936,  0.48575812,
        2.03911007, -1.10031333, -0.40622436,  0.24109638, -0.6489704 ,
       -1.97050145,  0.09435958, -0.91618468,  0.39386854, -0.93195056,
        1.21101961, -0.66223164, -0.34742302,  0.88112607, -1.15063722,
        0.18146311,  1.35918336, -1.64153013,  0.34284287,  0.40648783,
        0.99553769, -1.35064288, -1.24120905,  0.54834596,  0.32916065]), array([ 0.54155319, -

In [22]:
y # à tracer 

array([[-0.34285941, -2.07690054, -0.02012467, ..., -0.55421566,
         0.70420584,  0.3653095 ],
       [ 0.01546602, -2.23205831,  0.22764107, ...,  1.28610076,
        -1.19044683, -1.39997055],
       [-0.27099921,  0.43721934, -0.96957239, ..., -0.78443008,
         1.36211272,  1.4303267 ],
       ...,
       [ 0.51450932, -1.07085069, -1.79038001, ..., -1.84184646,
        -0.20222532,  0.68043279],
       [-1.33553995, -1.58544234, -0.24076355, ..., -3.07086435,
         0.7562848 ,  1.25141375],
       [-0.98886192,  1.51990591, -0.36214154, ...,  0.70992497,
         0.58586527, -3.68414091]])

In [30]:
y.shape

(100, 50)

In [63]:
# Exécution pour des données réelles normalisées

np.random.seed(42)

# Conversion en tableau NumPy
y = GDP.values.T  # Transpose pour avoir (N, T)
print(y.shape)  # Doit afficher (69, 27)

K = 3  # Nombre de clusters
num_iter = 500  # Nombre d'itérations pour le MCMC

# Estimation MCMC
S_est, theta_est, eta_est = mcmc(y, K, num_iter)

print("Classification estimée : ", S_est)
print("Paramètres des groupes : ", theta_est)
print("Paramètres eta : ", eta_est)

(69, 27)
Classification estimée :  [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
Paramètres des groupes :  [array([-0.3042521 , -0.11670463, -0.3473727 , -0.53600333, -0.30915443,
       -0.27934743,  0.89081317, -0.89444088,  1.9536459 , -0.66514828,
        0.86910079, -1.42708004, -0.66648318, -0.22104584, -0.86772683,
        0.57914114,  0.56361525, -0.70822635, -1.81519983,  0.62686454,
        0.07987199, -0.48536948, -0.57860071, -0.70380732,  0.84741842,
       -0.33620734,  1.37977526]), array([0.03085638, 0.06074844, 0.11206989, 0.13884558, 0.16606907,
       0.20141824, 0.25704613, 0.31076316, 0.3797597 , 0.45083692,
       0.48009911, 0.42329402, 0.47721469, 0.51991825, 0.53946059,
       0.57033192, 0.60810629, 0.64955105, 0.69165072, 0.75201168,
       0.8053718 , 0.84722232, 0.75697621, 0.88197098, 0.95204807,
       0.98845817, 0.41973416]), array([ 2.37127377,  0.66286393,  

In [ ]:
# Exécution pour des données réelles normalisées sur l'ensemble

np.random.seed(42)

# Conversion en tableau NumPy
y_global = GDP_global.values.T  # Transpose pour avoir (N, T)

K = 3  # Nombre de clusters
num_iter = 500  # Nombre d'itérations pour le MCMC

# Estimation MCMC
S_est, theta_est, eta_est = mcmc(y_global, K, num_iter)

print("Classification estimée : ", S_est)
print("Paramètres des groupes : ", theta_est)
print("Paramètres eta : ", eta_est)

Classification estimée :  [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
Paramètres des groupes :  [array([ 1.74169077e+00, -3.81474571e-01,  4.25480967e-01,  1.09195970e+00,
       -1.89837742e+00, -8.37492785e-01, -1.23806434e-03, -1.18768745e+00,
       -6.75776988e-01, -1.79102207e+00, -9.14564488e-01, -8.18783113e-01,
        1.62320855e+00,  6.30259872e-01, -1.36151115e+00, -5.45606038e-01,
        3.16015286e-01, -7.85183851e-01, -6.70778772e-01,  1.63083555e+00,
       -1.22885632e+00,  2.08056312e+00, -1.24419540e+00, -2.11153265e+00,
        1.62670924e+00,  2.98005680e-01, -1.55600629e-01]), array([ 3.86532725e-01, -1.44286948e+00, -5.56678995e-01,  9.53890045e-01,
       -6.86679973e-01, -7.04344431e-01,  1.15052783e+00, -5.36561176e-02,
        3.52715559e-01, -1.42268760e+00,  1.60169876e-03,  1.38842350e+00,
       -1.80436284e+00, -9.76998604e-01,  1.21490237e+00,  3.84131575e-

## Deuxième exemple avec un modèle ARMA(p, q)

In [79]:
from statsmodels.tsa.arima.model import ARIMA

In [80]:
def sample_S_arma(y, theta, eta, p, q):
    """
    Échantillonne les indicateurs de groupe S.
    y : séries temporelles
    theta : paramètres ARMA pour chaque groupe
    eta : probabilités de groupe
    """
    N = len(y)
    K = len(eta)
    log_probs = np.zeros((N, K))
    
    for k in range(K):
        for i in range(N):
            log_probs[i, k] = np.log(eta[k]) + arma_log_likelihood(y[i], theta[k], p, q)
    
    # Normalisation pour obtenir des probabilités
    probs = np.exp(log_probs - log_probs.max(axis=1, keepdims=True))
    probs /= probs.sum(axis=1, keepdims=True)

    # Échantillonnage des groupes
    S = np.array([np.random.choice(K, p=probs[i]) for i in range(N)])
    return S

def update_theta_arma(y, S, K, p, q):
    """
    Met à jour les paramètres ARMA pour chaque groupe.
    """
    theta = []
    for k in range(K):
        group_data = y[S == k]  # Séries assignées au groupe k
        theta_k = estimate_arma_params(group_data, p, q)
        theta.append(theta_k)
    return theta

def arma_log_likelihood(y, theta, p, q):
    """
    Calcule la vraisemblance logarithmique d'un modèle ARMA.
    y : série temporelle
    theta : paramètres ARMA
    """
    # Placeholder : remplacer par une vraie implémentation
    return -0.5 * np.sum((y - np.convolve(y, theta, mode='same'))**2)

def estimate_arma_params(y, p, q):
    """
    Estime les paramètres ARMA pour un ensemble de séries.
    """
    # Placeholder : remplacer par une vraie implémentation
    return np.random.randn(p + q)

In [77]:
def mcmc_arma(y, K, p, q, num_iterations, e0=1.0):
    """
    Exécute une chaîne MCMC pour estimer les paramètres du modèle ARMA avec structure d'ignorance pour eta.
    y : séries temporelles (array [N x T])
    K : nombre de groupes
    p : ordre AR
    q : ordre MA
    num_iterations : nombre d'itérations de MCMC
    e0 : hyperparamètre de la distribution Dirichlet pour eta
    """
    # Initialisation des paramètres
    eta, S, theta = initialize_model(y, K, e0)

    for iter in range(num_iterations):
        # Étape (a) : Échantillonnage des groupes S
        S = sample_S_arma(y, theta, eta, p, q)

        # Étape (b.1) : Mise à jour des paramètres ARMA pour chaque groupe
        theta = update_theta_arma(y, S, K, p, q)

        # Étape (b.2) : Mise à jour des probabilités de groupe eta
        eta = update_eta(S, K, e0)

        # Diagnostics ou suivi des progrès
        if iter % 100 == 0:
            print(f"Iteration {iter}: eta = {eta}")

    return S, theta, eta


In [78]:
# Exécution pour des données réelles normalisées sur l'ensemble
np.random.seed(42)

# Conversion en tableau NumPy
y_global = GDP_global.values.T  # Transpose pour avoir (N, T)

K = 3  # Nombre de clusters
num_iter = 500  # Nombre d'itérations pour le MCMC

# Estimation MCMC
S_est, theta_est, eta_est = mcmc_arma(y_global, K, 2, 0, num_iter)

print("Classification estimée : ", S_est)
print("Paramètres des groupes : ", theta_est)
print("Paramètres eta : ", eta_est)

Iteration 0: eta = [0.13734109 0.56517378 0.29748513]
Iteration 100: eta = [0.2842857  0.13624712 0.57946718]
Iteration 200: eta = [0.29039462 0.63451304 0.07509233]
Iteration 300: eta = [0.30630294 0.31722039 0.37647666]
Iteration 400: eta = [0.57922404 0.02641062 0.39436534]
Classification estimée :  [1 1 0 0 1 1 0 0 0 2 0 2 0 2 1 0 0 2 2 2 1 0 0 0 0 1 1 2 0 0 0 2 2 0 1 1 1
 0 1 0 2 0 1 1 2 1 1 2 0 1 0 2 0 1 0 0 1 1 1 0 1 0 0 2 2 0 1 1 2]
Paramètres des groupes :  [array([-0.47449519,  1.40787666]), array([-0.42231031,  0.51687491]), array([-1.07354007, -1.26681498])]
Paramètres eta :  [0.37201194 0.27647577 0.35151229]


In [81]:
# Exécution pour des données réelles normalisées sur l'ensemble
np.random.seed(42)

# Conversion en tableau NumPy
y = GDP.values.T  # Transpose pour avoir (N, T)

K = 3  # Nombre de clusters
num_iter = 500  # Nombre d'itérations pour le MCMC

# Estimation MCMC
S_est, theta_est, eta_est = mcmc_arma(y, K, 2, 0, num_iter)

print("Classification estimée : ", S_est)
print("Paramètres des groupes : ", theta_est)
print("Paramètres eta : ", eta_est)

Iteration 0: eta = [0.44378555 0.52489075 0.0313237 ]
Iteration 100: eta = [0.10782093 0.81020489 0.08197418]
Iteration 200: eta = [0.00443055 0.99327433 0.00229511]
Iteration 300: eta = [0.02644586 0.06236321 0.91119093]
Iteration 400: eta = [0.12910007 0.86849781 0.00240211]
Classification estimée :  [2 2 2 2 2 2 2 2 2 2 2 2 0 2 2 2 2 2 2 2 2 2 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 2 2 2 0 2 2 2 2 2]
Paramètres des groupes :  [array([-0.26180907,  0.35715229]), array([ 0.13350516, -0.22636144]), array([-0.57130212, -1.56560738])]
Paramètres eta :  [0.05955011 0.01606546 0.92438443]


## Estimer le nombre optimal de clusters K

en utilisant une prior uniforme pour M_k : Pr(MK)= 1/Kmax
 

Etape 1 : Bridge Sampling

In [88]:
import numpy as np
from scipy.stats import dirichlet

def bridge_sampling(log_posterior, log_prior, proposal_density, posterior_samples, proposal_samples):
    """
    Implémente la méthode de bridge sampling pour estimer la vraisemblance marginale.
    log_posterior: fonction qui calcule le log de la densité postérieure
    log_prior: fonction qui calcule le log de la prior
    proposal_density: fonction qui calcule la densité de la proposition
    posterior_samples: échantillons de la distribution postérieure (MCMC)
    proposal_samples: échantillons de la distribution de proposition (iid)
    """
    # Calcul des poids de bridge
    log_w_posterior = log_posterior(posterior_samples) + log_prior(posterior_samples) - proposal_density(posterior_samples)
    log_w_proposal = log_posterior(proposal_samples) + log_prior(proposal_samples) - proposal_density(proposal_samples)
    
    w_posterior = np.exp(log_w_posterior)
    w_proposal = np.exp(log_w_proposal)
    
    Z_ratio = np.sum(w_posterior) / np.sum(w_proposal)
    marginal_likelihood = np.sum(w_posterior / (Z_ratio + w_posterior))
    
    return np.log(marginal_likelihood)


Étape 2 : Intégration avec MCMC et choix de K

In [89]:
def compute_marginal_likelihood(y, K, num_iterations, p, q, e0=1.0):
    """
    Calcule la vraisemblance marginale pour un nombre donné de clusters K.
    """
    # Étape 1 : Échantillonnage MCMC pour le modèle avec K clusters
    S, theta, eta = mcmc_arma(y, K, p, q, num_iterations, e0)
    
    # Étape 2 : Définir les fonctions de log posterior, prior et proposition
    def log_posterior(params):
        # Placeholder : remplacez par le calcul du log posterior
        return -0.5 * np.sum(params**2)
    
    def log_prior(params):
        # Placeholder : prior sur les paramètres
        return -0.5 * np.sum(params**2)
    
    def proposal_density(params):
        # Placeholder : densité de proposition gaussienne
        return -0.5 * np.sum(params**2)
    
    # Étape 3 : Générer des échantillons pour bridge sampling
    posterior_samples = np.random.randn(1000, len(theta))  # Échantillons postérieurs simulés
    proposal_samples = np.random.randn(1000, len(theta))  # Échantillons de proposition
    
    # Étape 4 : Bridge sampling
    log_marginal_likelihood = bridge_sampling(log_posterior, log_prior, proposal_density, posterior_samples, proposal_samples)
    return log_marginal_likelihood


Étape 3 : Estimation du nombre optimal de clusters

In [90]:
def choose_optimal_clusters(y, K_max, num_iterations, p, q, e0=1.0):
    """
    Choisit le nombre optimal de clusters en utilisant la vraisemblance marginale.
    """
    log_marginal_likelihoods = []
    
    for K in range(1, K_max + 1):
        log_ml = compute_marginal_likelihood(y, K, num_iterations, p, q, e0)
        log_marginal_likelihoods.append(log_ml)
        print(f"K={K}, log marginal likelihood = {log_ml}")
    
    optimal_K = np.argmax(log_marginal_likelihoods) + 1
    return optimal_K, log_marginal_likelihoods


In [91]:
def choose_optimal_clusters_with_prior(y, K_max, num_iterations, p, q, e0=1.0):
    """
    Choisit le nombre optimal de clusters en utilisant la vraisemblance marginale
    avec une prior explicite sur les modèles.
    """
    log_marginal_likelihoods = []
    prior_probabilities = [1 / K_max] * K_max  # Prior uniforme

    for K in range(1, K_max + 1):
        log_ml = compute_marginal_likelihood(y, K, num_iterations, p, q, e0)
        log_prior = np.log(prior_probabilities[K - 1])
        log_marginal_likelihoods.append(log_ml + log_prior)
        print(f"K={K}, log marginal likelihood + log prior = {log_ml + log_prior}")

    optimal_K = np.argmax(log_marginal_likelihoods) + 1
    return optimal_K, log_marginal_likelihoods


Utilisation

In [87]:
# Exécution pour des données réelles normalisées sur l'ensemble
np.random.seed(42)

# Conversion en tableau NumPy
y_global = GDP_global.values.T  # Transpose pour avoir (N, T)
K_max = 10
num_iterations = 1000
p, q = 1, 1

optimal_K, log_marginal_likelihoods = choose_optimal_clusters_with_prior(y_global, K_max, num_iterations, p, q)
print(f"Nombre optimal de clusters : {optimal_K}")


Iteration 0: eta = [1.]
Iteration 100: eta = [1.]
Iteration 200: eta = [1.]
Iteration 300: eta = [1.]
Iteration 400: eta = [1.]
Iteration 500: eta = [1.]
Iteration 600: eta = [1.]
Iteration 700: eta = [1.]
Iteration 800: eta = [1.]
Iteration 900: eta = [1.]
K=1, log marginal likelihood + log prior = -506.63037858262845
Iteration 0: eta = [0.26478075 0.73521925]
Iteration 100: eta = [0.12587618 0.87412382]
Iteration 200: eta = [0.38767076 0.61232924]
Iteration 300: eta = [0.79586043 0.20413957]
Iteration 400: eta = [0.57810088 0.42189912]
Iteration 500: eta = [0.75718219 0.24281781]
Iteration 600: eta = [0.68836394 0.31163606]
Iteration 700: eta = [0.41571736 0.58428264]
Iteration 800: eta = [0.27223669 0.72776331]
Iteration 900: eta = [0.64557373 0.35442627]


C:\Users\hp\AppData\Local\Temp\ipykernel_12956\3997777796.py:20: RuntimeWarning: invalid value encountered in scalar divide
  Z_ratio = np.sum(w_posterior) / np.sum(w_proposal)


K=2, log marginal likelihood + log prior = nan
Iteration 0: eta = [0.46381527 0.45736827 0.07881646]
Iteration 100: eta = [0.06409691 0.69500333 0.24089976]
Iteration 200: eta = [0.6727733  0.31035314 0.01687356]
Iteration 300: eta = [0.17436685 0.25593848 0.56969467]
Iteration 400: eta = [0.00616875 0.56747018 0.42636107]
Iteration 500: eta = [0.46233875 0.02578251 0.51187874]
Iteration 600: eta = [0.4906374  0.1209427  0.38841989]
Iteration 700: eta = [0.00578596 0.48739303 0.50682101]
Iteration 800: eta = [0.08652585 0.59293892 0.32053523]
Iteration 900: eta = [0.45748703 0.15172928 0.39078368]
K=3, log marginal likelihood + log prior = nan
Iteration 0: eta = [0.1210421  0.26068525 0.18595499 0.43231766]
Iteration 100: eta = [0.08292567 0.50142407 0.2014924  0.21415787]
Iteration 200: eta = [0.37774112 0.06067071 0.16262365 0.39896452]
Iteration 300: eta = [0.13146601 0.33642341 0.52084928 0.0112613 ]
Iteration 400: eta = [0.28293227 0.16444583 0.29657572 0.25604618]
Iteration 500: 